In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
from datasets import load_dataset # For other datasets
from sentence_transformers import SentenceTransformer #to encode text to vectors
from qdrant_client import QdrantClient, models

import os
import time
import torch

from tqdm.auto import tqdm

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device != 'cuda':
    print('Sorry no cuda.')
else:
    print('Using CUDA-GPU')
    
encoderModel = SentenceTransformer('all-MiniLM-L6-v2', device=device) # Model to create embeddings

Using CUDA-GPU


In [3]:
query = 'Appendicectomy procedure'
xq = encoderModel.encode(query)
print(f'''The dimension of the vector is {xq.shape}''')
xq

The dimension of the vector is (384,)


array([-1.41302152e-02,  5.11170365e-02, -7.05268141e-03, -4.34320606e-02,
       -7.33532757e-02, -9.40867364e-02,  4.23129415e-03,  2.10136212e-02,
       -1.14401393e-02,  4.96104881e-02,  7.46712685e-02,  4.05386835e-02,
       -1.19351493e-02, -2.58628670e-02, -2.21027061e-02, -1.26899211e-02,
        7.78764719e-03, -7.04822391e-02, -1.27317850e-02,  2.95674633e-02,
        8.93106684e-03,  4.02070722e-03,  4.30439562e-02, -1.39735267e-02,
        6.98572919e-02, -8.89082775e-02, -5.86595722e-02,  7.88074732e-02,
        5.89536084e-03, -2.64904313e-02,  5.06862104e-02, -4.56224605e-02,
       -1.56551059e-02, -3.91518660e-02,  8.18973854e-02, -1.62047090e-03,
        2.55833939e-02,  6.93021342e-02, -4.90610115e-03,  1.30840018e-03,
        4.78595607e-02, -1.97446961e-02, -2.30856631e-02,  2.22700536e-02,
        2.88418438e-02, -3.77949141e-02, -6.07232861e-02, -4.13567014e-02,
        3.97630297e-02,  5.00037670e-02, -3.59451771e-02, -6.08060360e-02,
       -2.40867324e-02,  

In [4]:
qdrant = QdrantClient("http://localhost:6333") 

In [5]:
collectionInfo =  None
try:
    collectionInfo = qdrant.get_collection(collection_name="HospitalCharges")
except:
    print('Exception found')
    
    
collectionInfo

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=0, indexed_vectors_count=0, points_count=0, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=384, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None), payload_schema={})

In [6]:
# Create collection to store questions

#Carefull not to recreate collection every time
if not collectionInfo:
    qdrant.recreate_collection(
        collection_name="HospitalCharges",
        vectors_config=models.VectorParams(
            size=encoderModel.get_sentence_embedding_dimension(), # Vector size is defined by used model
            distance=models.Distance.COSINE
        )
    )

In [ ]:
name_of_dir = '..\\..\\Data\\HospitalCharges\\'
# Storing list of all files 
# in the given directory in list_of_files 
list_of_files = filter( lambda x: os.path.isfile 
                       (os.path.join(name_of_dir, x)), 
                        os.listdir(name_of_dir) ) 
  
# Sort list of file names by size  
list_of_files = sorted( list_of_files, 
                        key =  lambda x: os.stat 
                       (os.path.join(name_of_dir, x)).st_size) 
  
# Iterate over sorted list of file  
# names and print them along with size one by one  
for name_of_file in list_of_files: 
    path_of_file = os.path.join(name_of_dir, name_of_file) 
    size_of_file  = os.stat(path_of_file).st_size  
    print(size_of_file/(1024*1024), 'MBytes -->', name_of_file) 

In [ ]:
#Read the file into a dataframe
for filename in filenames:
    if filename.endswith(".zip"):
        file2load = os.path.join(dirname, filename)
        print(file2load)

In [7]:
## Read the zip files containing Hosiptal Charge Data into a dataframe
df = pd.read_csv('..\\..\\Data\\HospitalCharges\\042104691_Marthas-Vineyard-Hospital_StandardCharges.zip', 
                 sep='|', encoding='latin', low_memory=False, on_bad_lines='warn')

df.rename(columns={'procedure': 'procedurename', 'contract': 'contractname'}, inplace=True)
(df.dtypes)

facility_id                    int64
facility_name                 object
procedurename                 object
code_type                     object
code                          object
min_ip_reimb                 float64
max_ip_reimb                 float64
min_op_reimb                 float64
max_op_reimb                 float64
ndc                           object
rev_code                      object
procedure_description         object
quantity                      object
payer                         object
contractname                  object
plan                          object
ip_price                     float64
ip_pricing_detail             object
ip_expected_reimbursement    float64
ip_xr_detail                  object
op_price                     float64
op_pricing_detail             object
op_expected_reimbursement    float64
op_xr_detail                  object
dtype: object

In [8]:
#Subset dataframe to include procedure_description and additonal fields for metadata
dfData = df[['facility_id', 'procedurename', 'procedure_description', 'payer', 'contractname', 'plan', 
               'ip_pricing_detail', 'ip_xr_detail', 'ip_price', 'ip_expected_reimbursement']]

dfData = dfData.fillna('')

In [9]:
dfData

,facility_id,procedurename,procedure_description,payer,contractname,plan,ip_pricing_detail,ip_xr_detail,ip_price,ip_expected_reimbursement
0,1011999,331035,SET CHANNEL SEAL ENDOSCOPIC INSTRUMENT ROD LEN...,HUMANA [1012],HB MVH MEDICARE [434],HUMANA HMO MEDICARE REPLACEMENT [101201],Inv Loc: 3061000003; from OR location 3061000003,No historical claim,0.00,0.00
1,1011999,30000736,Vitamin K,CIGNA [1006],HB MVH CIGNA HMO / PPO [273],CIGNA HMO POS [100602],FSC: 296; BFG: 52,Inpatient / Outpatient PAF; 91.4% Billed,184.00,168.18
2,1011999,31000821,Prenatal Known Familial Mutation (Includes Mcc),TUFTS HEALTH PLAN [170001],HB MVH TUFTS HMO / PPO / CARELINK [327],TUFTS CARELINK PPO [17000101],FSC: 48; Median of: 1,Inpatient / Outpatient HMO PAF; 74.8% Billed,1521.00,1137.71
3,1011999,528969,HAND PLATE 0.8MM 10 HOLE COMPRESSION STRAIGHT,TUFTS HEALTH PLAN [170001],HB MVH TUFTS HMO / PPO / CARELINK [327],TUFTS CARELINK PPO [17000101],Inv Loc: 3061000003; from OR location 3061000003,Inpatient / Outpatient PPO PAF; 76.2% Billed,1338.00,1019.56
4,1011999,119834,HYDROMORPHONE 1 MG/ML INJECTION SYRINGE,TUFTS HEALTH PUBLIC PLANS [1013],HB MVH MEDICAID [254],TUFTS PUBLIC PLANS MASSHEALTH TOGETHER MCO [10...,PHR: 4080000631; CHG: 408000007,No contract line matched,4.83,0.00
...,...,...,...,...,...,...,...,...,...,...
444501,1011999,97371,VANCOMYCIN 750 MG INTRAVENOUS SOLUTION,HARVARD PILGRIM [120001],HB MVH HPHC HMO / POS / PPO [329],HPHC QUALITY LIMITED NTWK HMO [12000101],PHR: 4080000631; CHG: 408000007,Inpatient / Outpatient PAF; 78.18% Billed,26.62,20.81
444502,1011999,76000021,Debride/Remove FB Skin/Muscle/Bone,HEALTH NEW ENGLAND [1034],HB MVH MEDICAID [254],HEALTH NEW ENGLAND CONNECTOR CARE [103102],FSC: 296; BFG: 52,No historical claim; No contract line matched,2025.00,0.00
444503,1011999,76001906,Stab Phlebetomy <10 Incisions,HUMANA [1012],HB MGB HUMANA [39],HUMANA PPO POS [101202],FSC: 48; Median of: 1,Inpatient / Outpatient PAF; 92% Billed,2548.00,2344.16
444504,1011999,460867,CATHETER FOLEY 16 FRIN BARDEX IC BACTI GUARD A...,AETNA [1001],HB MGB AETNA INDEMNITY [31],AETNA INDEMNITY [100102],Inv Loc: 3061000003; from OR location 3061000003,No historical claim; Inpatient / Outpatient In...,0.00,0.00


In [10]:
#convert data frame into json_list
json_list = [dict(v) for _, v in dfData.iterrows()]

In [11]:
json_list

[{'facility_id': 1011999,
  'procedurename': '331035',
  'procedure_description': 'SET CHANNEL SEAL ENDOSCOPIC INSTRUMENT ROD LENS SCOPE OUTFLOW MYOSURE BX/10EA',
  'payer': 'HUMANA [1012]',
  'contractname': 'HB MVH MEDICARE [434]',
  'plan': 'HUMANA HMO MEDICARE REPLACEMENT [101201]',
  'ip_pricing_detail': 'Inv Loc: 3061000003; from OR location 3061000003',
  'ip_xr_detail': 'No historical claim',
  'ip_price': 0.0,
  'ip_expected_reimbursement': 0.0},
 {'facility_id': 1011999,
  'procedurename': '30000736',
  'procedure_description': 'Vitamin K',
  'payer': 'CIGNA [1006]',
  'contractname': 'HB MVH CIGNA HMO / PPO [273]',
  'plan': 'CIGNA HMO POS [100602]',
  'ip_pricing_detail': 'FSC: 296; BFG: 52',
  'ip_xr_detail': 'Inpatient / Outpatient PAF; 91.4% Billed',
  'ip_price': 184.0,
  'ip_expected_reimbursement': 168.18},
 {'facility_id': 1011999,
  'procedurename': '31000821',
  'procedure_description': 'Prenatal Known Familial Mutation (Includes Mcc)',
  'payer': 'TUFTS HEALTH PLA

In [12]:
# Create Embeddings (Vectorize) and Upsert to Qdrant
batch_size=200
vector_limit=10000000

json_list = json_list[:vector_limit]

for i in tqdm(range(0, len(json_list), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(json_list))
    
    qdrant.upsert(
        collection_name="HospitalCharges",
        points=[
            models.PointStruct(
                id=idx+i_end, #ID for the record in the collection
                vector=encoderModel.encode(doc["procedure_description"]).tolist(), #encoded vector text
                payload=doc #meta data for the records
            ) for idx, doc in enumerate(json_list[i:i_end])
            ]
        )

  0%|          | 0/2223 [00:00<?, ?it/s]

In [13]:
# Let's now search for something
query = 'Appendicectomy procedure'

hits = qdrant.search(
    collection_name="HospitalCharges",
    query_vector=encoderModel.encode(query).tolist(),
    limit=30
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'contractname': 'HB MVH MEDICARE [434]', 'facility_id': 1011999, 'ip_expected_reimbursement': 0.0, 'ip_price': 2036.0, 'ip_pricing_detail': 'FSC: 296; BFG: 52', 'ip_xr_detail': 'No contract line matched', 'payer': 'UNITED HEALTHCARE [1016]', 'plan': 'UNITED SCO ERICKSON COMMUNITY MEDICARE REPLACEMENT [101601]', 'procedure_description': 'Laparoscopic Appendectomy', 'procedurename': '98301068'} score: 0.75502884
{'contractname': 'HB MVH MEDICAID [254]', 'facility_id': 1011999, 'ip_expected_reimbursement': 0.0, 'ip_price': 2036.0, 'ip_pricing_detail': 'FSC: 296; BFG: 52', 'ip_xr_detail': 'No contract line matched', 'payer': 'HEALTH NEW ENGLAND [1034]', 'plan': 'HEALTH NEW ENGLAND CONNECTOR CARE [103102]', 'procedure_description': 'Laparoscopic Appendectomy', 'procedurename': '98301068'} score: 0.75502884
{'contractname': 'HB MVH MEDICARE [434]', 'facility_id': 1011999, 'ip_expected_reimbursement': 0.0, 'ip_price': 509.0, 'ip_pricing_detail': 'FSC: 48; Median of: 1', 'ip_xr_detail': 'No h

In [14]:
dfResults = pd.DataFrame([hit.payload for hit in hits])

In [15]:
dfResults

,contractname,facility_id,ip_expected_reimbursement,ip_price,ip_pricing_detail,ip_xr_detail,payer,plan,procedure_description,procedurename
0,HB MVH MEDICARE [434],1011999,0.00,2036.0,FSC: 296; BFG: 52,No contract line matched,UNITED HEALTHCARE [1016],UNITED SCO ERICKSON COMMUNITY MEDICARE REPLACE...,Laparoscopic Appendectomy,98301068
1,HB MVH MEDICAID [254],1011999,0.00,2036.0,FSC: 296; BFG: 52,No contract line matched,HEALTH NEW ENGLAND [1034],HEALTH NEW ENGLAND CONNECTOR CARE [103102],Laparoscopic Appendectomy,98301068
2,HB MVH MEDICARE [434],1011999,0.00,509.0,FSC: 48; Median of: 1,No historical claim; No contract line matched,TUFTS HEALTH PLAN [170001],TUFTS MEDICARE PREFERRED HMO REPLACEMENT [1700...,Escharotomy Ea Add Incision,76000209
3,HB MVH MEDICAID [254],1011999,0.00,509.0,FSC: 48; Median of: 1,No historical claim; No contract line matched,HEALTH NEW ENGLAND [1034],HEALTH NEW ENGLAND CONNECTOR CARE [103102],Escharotomy Ea Add Incision,76000209
4,HB MVH TUFTS HMO / PPO / CARELINK [327],1011999,34698.22,46388.0,FSC: 48; Median of: 1,No historical claim; Inpatient / Outpatient HM...,TUFTS HEALTH PLAN [170001],TUFTS CARELINK PPO [17000101],Excision Infected Graft Abdomen,36000163
5,HB MVH BLUE CROSS HMO [316],1011999,31441.79,46388.0,FSC: 48; Median of: 1,No historical claim; Inpatient PAF; 67.78% Billed,BLUE CROSS BLUE SHIELD [110001],BLUE CROSS MA HMO POS [11000103],Excision Infected Graft Abdomen,36000163
6,HB MVH MGBHP COMMERCIAL PPO [417],1011999,34837.39,46388.0,FSC: 48; Median of: 1,No historical claim; Inpatient / Outpatient PA...,MGB HEALTH PLAN [150001],MGBHP PPO [15000107],Excision Infected Graft Abdomen,36000163
7,HB MVH MEDICARE [434],1011999,0.00,46388.0,FSC: 48; Median of: 1,No historical claim; No contract line matched,SENIOR WHOLE HEALTH [1014],SENIOR WHOLE HEALTH MEDICARE REPLACEMENT [101401],Excision Infected Graft Abdomen,36000163
8,HB MGB AETNA INDEMNITY [31],1011999,46388.00,46388.0,FSC: 48; Median of: 1,No historical claim; Inpatient / Outpatient In...,AETNA [1001],AETNA INDEMNITY [100102],Excision Infected Graft Abdomen,36000163
9,HB MVH AETNA [330],1011999,38965.92,46388.0,FSC: 48; Median of: 1,No historical claim; Inpatient PAF; 84% Billed,AETNA [1001],AETNA HMO POS EPO [100101],Excision Infected Graft Abdomen,36000163


In [16]:
query="Admin of Flumist Influenza Vaccine"
hits = qdrant.search(
    collection_name="HospitalCharges",
    query_vector=encoderModel.encode(query).tolist(),
    limit=300
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'contractname': 'HB MVH HPHC HMO / POS / PPO [329]', 'facility_id': 1011999, 'ip_expected_reimbursement': 22.67, 'ip_price': 29.0, 'ip_pricing_detail': 'FSC: 296; BFG: 52', 'ip_xr_detail': 'Inpatient / Outpatient PAF; 78.18% Billed', 'payer': 'HARVARD PILGRIM [120001]', 'plan': 'HPHC QUALITY LIMITED NTWK HMO [12000101]', 'procedure_description': 'Admin of Influenza Vaccine', 'procedurename': '77100008'} score: 0.917377
{'contractname': 'HB MVH MEDICARE [434]', 'facility_id': 1011999, 'ip_expected_reimbursement': 0.0, 'ip_price': 30.0, 'ip_pricing_detail': 'FSC: 296; BFG: 52', 'ip_xr_detail': 'No contract line matched', 'payer': 'AETNA [1001]', 'plan': 'AETNA PPO MEDICARE REPLACEMENT [100103]', 'procedure_description': 'Admin of Influenza Vaccine', 'procedurename': '77100007'} score: 0.917377
{'contractname': 'HB MVH MEDICARE [434]', 'facility_id': 1011999, 'ip_expected_reimbursement': 0.0, 'ip_price': 30.0, 'ip_pricing_detail': 'FSC: 296; BFG: 52', 'ip_xr_detail': 'No contract line ma

In [17]:
hit

ScoredPoint(id=93848, version=468, score=0.47719756, payload={'contractname': 'HB MVH PHCS [332]', 'facility_id': 1011999, 'ip_expected_reimbursement': 817.18, 'ip_price': 898.0, 'ip_pricing_detail': 'FSC: 48; Median of: 1', 'ip_xr_detail': 'Inpatient PAF; 91% Billed', 'payer': 'PHCS MULTIPLAN [1022]', 'plan': 'GENERIC PHCS MULTIPLAN [102201]', 'procedure_description': 'Infectious Agent Dna/Rna Influenza 1st 2 Types', 'procedurename': '30001285'}, vector=None, shard_key=None)

In [20]:
dfResults = pd.DataFrame([hit.payload for hit in hits])
dfResults['query_score'] = [hit.score for hit in hits]

In [21]:
dfResults

,contractname,facility_id,ip_expected_reimbursement,ip_price,ip_pricing_detail,ip_xr_detail,payer,plan,procedure_description,procedurename,query_score
0,HB MVH HPHC HMO / POS / PPO [329],1011999,22.67,29.0,FSC: 296; BFG: 52,Inpatient / Outpatient PAF; 78.18% Billed,HARVARD PILGRIM [120001],HPHC QUALITY LIMITED NTWK HMO [12000101],Admin of Influenza Vaccine,77100008,0.917377
1,HB MVH MEDICARE [434],1011999,0.00,30.0,FSC: 296; BFG: 52,No contract line matched,AETNA [1001],AETNA PPO MEDICARE REPLACEMENT [100103],Admin of Influenza Vaccine,77100007,0.917377
2,HB MVH MEDICARE [434],1011999,0.00,30.0,FSC: 296; BFG: 52,No contract line matched,HUMANA [1012],HUMANA HMO MEDICARE REPLACEMENT [101201],Admin of Influenza Vaccine,77100007,0.917377
3,HB MVH MEDICARE [434],1011999,0.00,30.0,FSC: 296; BFG: 52,No contract line matched,TUFTS HEALTH PLAN [170001],TUFTS MEDICARE PREFERRED HMO REPLACEMENT [1700...,Admin of Influenza Vaccine,77100007,0.917377
4,HB MVH MEDICARE [434],1011999,0.00,29.0,FSC: 296; BFG: 52,No contract line matched,UNITED HEALTHCARE [1016],UNITED SCO ERICKSON COMMUNITY MEDICARE REPLACE...,Admin of Influenza Vaccine,77100008,0.917377
...,...,...,...,...,...,...,...,...,...,...,...
295,HB MVH CIGNA MEDICARE REPLACEMENT [416],1011999,619.62,898.0,FSC: 48; Median of: 1,Inpatient / Outpatient PAF; 69% Billed,CIGNA [1006],CIGNA PPO MEDICARE REPLACEMENT [100616],Infectious Agent Dna/Rna Influenza 1st 2 Types,30001285,0.477198
296,HB MVH MEDICAID [254],1011999,898.00,898.0,FSC: 48; Median of: 1,No contract line matched,MASSHEALTH [3001],MASSHEALTH LIMITED [300105],Infectious Agent Dna/Rna Influenza 1st 2 Types,30001285,0.477198
297,HB MVH MEDICARE [434],1011999,0.00,898.0,FSC: 48; Median of: 1,No contract line matched,UNITED HEALTHCARE [1016],UNITED SCO ERICKSON COMMUNITY MEDICARE REPLACE...,Infectious Agent Dna/Rna Influenza 1st 2 Types,30001285,0.477198
298,HB MVH TUFTS HMO / PPO / CARELINK [327],1011999,684.28,898.0,FSC: 48; Median of: 1,Inpatient / Outpatient PPO PAF; 76.2% Billed,CIGNA [1006],CIGNA CARELINK PPO [100601],Infectious Agent Dna/Rna Influenza 1st 2 Types,30001285,0.477198
